# Avito Demand Prediction Challenge

Контест Avito по предсказанию успешности объявления, основываясь на его тексте и изображениях.

В рамках курса "Автоматическая обработка естественного языка" контест решали: 
* Дима Татаринов, БКЛ152,
* Даша Максимова, БКЛ151.


P.S.

К сожалению, при написании отчета, код обнулился и его пришлось запускать заново. Из-за чего большинство выводов нет.

## 1. Данные

```
!mkdir data
!gsutil cp -r gs://python-ml-hse/avito/data/data .
```
или

```
!gsutil cp -r gs://python-ml-hse/avito/data/data/train.csv ./data/train.csv
!gsutil cp -r gs://python-ml-hse/avito/data/data/test.csv ./data/test.csv
```

### 1.1. Загрузка

In [ ]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 500)

In [ ]:
data = pd.read_csv("./data/train.csv")
data.head()

В датасете имеются такие колонки-фичи:

* `item_id` — Ad id.
* `user_id` — User id.
* `region` — Ad region.
* `city` — Ad city.
* `parent_category_name` — Top level ad category as classified by Avito's ad model.
* `category_name` — Fine grain ad category as classified by Avito's ad model.
* `param_1` — Optional parameter from Avito's ad model.
* `param_2` — Optional parameter from Avito's ad model.
* `param_3` — Optional parameter from Avito's ad model.
* `title` — Ad title.
* `description` — Ad description.
* `price` — Ad price.
* `item_seq_number` — Ad sequential number for user.
* `activation_date`— Date ad was placed.
* `user_type` — User type.
* `image` — Id code of image. Ties to a jpg file in train_jpg. Not every ad has an image.
* `image_top_1` — Avito's classification code for the image.
* `deal_probability` — The target variable. This is the likelihood that an ad actually sold something. It's not possible to verify every transaction with certainty, so this column's value can be any float from zero to one.


Мы уберём следующие из них:
* `item_id`, `user_id` — это бесполезная для нас информация,
* `city` — кажется слишком мелким делением,
* `title` — у нас есть более содержательные тексты,
* `param_1`, `param_2`, `param_3` — опциональные и не всегда присутствующие параметры,
* `activation_date`, `item_seq_number` — тоже выглядит бесполезной,
* `image`, `image_top_1` — мы сконцентрируемся на текстах и категориальных переменных, а не на изображениях.

In [ ]:
cols_to_drop = ["item_id", "user_id", "city", "param_1", "param_2", "param_3", "title",
    "activation_date", "item_seq_number", "image", "image_top_1"]
data = data.drop(labels=cols_to_drop, axis=1)

In [ ]:
data.head()

Категориальные признаки закодируем (преобразуем в числовой вид):

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
parent_category = LabelEncoder()
parent_category.fit(data["parent_category_name"])
data["parent_category_name"] = parent_category.transform(data["parent_category_name"])

In [ ]:
category = LabelEncoder()
category.fit(data["category_name"])
data["category_name"] = category.transform(data["category_name"])

In [ ]:
user_type = LabelEncoder()
user_type.fit(data["user_type"])
data["user_type"] = user_type.transform(data["user_type"])

In [ ]:
region = LabelEncoder()
region.fit(data["region"])
data["region"] = region.transform(data["region"])

In [ ]:
data = data.dropna()

Так датасет выглядит сейчас:

In [ ]:
data.head()

### 1.2. Предобработка: тексты

Экспериментальным путём выяснено, что некоторые объявления пустые. Чтобы об них ничего не ломалось, запустим `fillna()`.

In [ ]:
data["description"].fillna("", inplace=True)

Тексты сначала приведём к нижнему регистру, а затем посчитаем количество токенов и приведём всё к леммам. По пути уберём стоп-слова. Это можно сделать так (Данный процесс занимает очень много времени, поэтому при выгрузки данных препроцессинг ограничен):

```python
from nltk import word_tokenize
from pymystem3 import Mystem
from nltk.corpus import stopwords

mystem = Mystem()


def count_words(text):
    try:
        len_words = len(word_tokenize(text))
    except:
        len_words = 0
    return len_words

def do_lemmas(text):
    try:
        stops = stopwords.words("russian")
        lemmas = [lemma for lemma in mystem.lemmatize(text) if lemma not in stops]
        return lemmas
    except:
        return ""
        
data_new["word_count"] = data_new["description"].apply(count_words)
data_new["lemmas"] = data_new["description"].apply(do_lemmas)
```

Используем модуль casual_tokenize из библиотеки nltk для токенизации текста.

Чистка текста от знаков препинания с помощью втроенной функции Python .isalpha()

In [ ]:
from nltk import casual_tokenize

In [ ]:
def tokenize(text):
    tokens = casual_tokenize(str(text))
    clean_stuff = [word.lower() for word in tokens if word.isalpha()]
    line = " ".join(clean_stuff)
    return line

In [ ]:
%%time
data["description"] = data["description"].apply(tokenize)

### 1.3. TF-IDF текстов + отделение фич

Для начала выделим целевую переменную и все категориальные и количественные фичи:

In [ ]:
cat_num_cols = ["region", "parent_category_name", "category_name", "price", "user_type"]
X_cat_num = data[cat_num_cols].values

In [ ]:
y = data["deal_probability"].values

Для стабильного решения проблемы с текстами, с помощью TF-IDF + SVM необходимо использовать PCA (метод главных компонентов, про который подробнее можно ознакоимться по ссылке: https://habr.com/post/304214/), или же ограничить размерность векторов получаемых методом TF-IDF для избавления от данных, которые влиют на результат не значительно. 

Применим TF-IDF:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

In [ ]:
tfidf = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents="unicode",
    analyzer="word",
    token_pattern=r"\w{1,}",
    stop_words=stopwords.words("russian"),
    max_features=10000
)

In [ ]:
%%time
tfidf.fit(data["description"].values)
X_texts = tfidf.transform(data["description"].values)

## 2. Объединение фич

На данном этапе неоходимо соединить все полученные нами катег. признаки с результатом работы TF-IDF.

In [ ]:
from scipy.sparse import hstack

In [ ]:
X = hstack((X_texts, X_cat_num))

In [ ]:
X.shape

Сохраняем на всякий случай:

In [ ]:
import os
from sklearn.externals import joblib

In [ ]:
try:
    os.mkdir("./models")
except:
    pass
joblib.dump(X, "./models/X.pkl")
joblib.dump(y, "./models/y.pkl")
joblib.dump(tfidf, "./models/tfidf.pkl")

## 3. Всякие разные алгоритмы

## 3.0. Подготовка

Для начала **разобьём выборку** на обучающую и тестовую:

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

**Создадим функцию ошибки** RMSE (root of mean squared error, корень из средней квадратичной ошибки), чтобы оценивать по ней:

In [ ]:
from sklearn.metrics import mean_squared_error, make_scorer
from math import sqrt

In [ ]:
def rmse_func(y_calc, y_test):
    rms = sqrt(mean_squared_error(y_actual, y_predicted))
    return rms

rmse = make_scorer(rmse_func, greater_is_better=False)

### 3.1. SVM

Классическое state of the art решение для задач на текстах — SVM на RBF-ядре.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

In [ ]:
%env JOBLIB_TEMP_FOLDER=/tmp

In [ ]:
params_svr = {"C": np.arange(0.1, 100.1, 0.1)}
svr = GridSearchCV(
    SVR(),
    param_grid=params_svc,
    scoring=rmse,
    cv=5,
    verbose=1,
    n_jobs=-1
)
svr.fit(X_train, y_train)

In [ ]:
print("SVR results:\n\t- best params: {}\n\t- best score: {}".format(svc.best_params_, svc.best_score_))

In [ ]:
result = svr.predict(y)
ids = data['item_id']
ids['deal_probability'] = result
ids.to_csv("submit2.csv",index=True,header=True)